In [ ]:
import plotly.express as px
import pandas as pd

df = pd.read_csv('../data/cleaned_security_incidents.csv')


In [2]:
print(df.country)


0                     Rwanda
1               Sierra Leone
2                     Rwanda
3                     Rwanda
4                     Rwanda
                ...         
3952                   Sudan
3953                DR Congo
3954                 Nigeria
3955    Syrian Arab Republic
3956                 Nigeria
Name: country, Length: 3957, dtype: object


In [12]:

# 1. Map Country to ISO codes (use your own `iso_codes` mapping if you already have it)
# Or use `pycountry` to auto-map (optional)
import pycountry

def get_iso_code(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except:
        return None

df['ISO'] = df['country'].apply(get_iso_code)

# 2. Filter by years in your range
df = df[(df['year'] >= 1997) & (df['year'] <= 2025)]

# 3. Group by Country and Year (summing Total.killed per country-year if needed)
df_grouped = df.groupby(['country', 'ISO', 'year'], as_index=False)['total.killed'].sum()

# 4. Plot the choropleth map
fig = px.choropleth(
    df_grouped,
    locations='ISO',
    locationmode='ISO-3',
    color='total.killed',
    hover_name='country',
    animation_frame='year',
    color_continuous_scale='Reds',
    title='Total Humanitarian Workers Killed by Country (1997–2025)'
)

fig.update_layout(
    title_x=0.5,
    geo=dict(showframe=False, showcoastlines=True),
    width=1000,
    height=600,
    template='plotly_white',
    coloraxis_colorbar=dict(title="Total Killed")
)

fig.update_traces(
    hovertemplate="<b>%{hovertext}</b><br>Total killed: %{z}<extra></extra>"
)

fig.show()


/opt/anaconda3/envs/dataviz5200/lib/python3.13/site-packages/_plotly_utils/basevalidators.py:2596: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

/opt/anaconda3/envs/dataviz5200/lib/python3.13/site-packages/_plotly_utils/basevalidators.py:2596: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# --- Step 1: Create your base choropleth map (you can reuse your earlier code) ---
fig = px.choropleth(
    df_grouped,
    locations='ISO',
    locationmode='ISO-3',
    color='total.killed',
    hover_name='country',
    animation_frame='year',
    color_continuous_scale='Reds',
    title='Total Humanitarian Workers Killed by Country (1997–2025)'
)


# --- Step 2: Select top 10 most affected events ---
top10 = df.sort_values(by='total.affected', ascending=False).head(10)

# --- Step 3: Add red circle markers on top ---
fig.add_trace(go.Scattergeo(
    lon=top10['longitude'],
    lat=top10['latitude'],
    text=top10['country'],  # optional: text when hovering
    marker=dict(
        size=12,
        color='red',
        opacity=0.8,
        symbol='circle'
    ),
    name='Top 10 Events'
))

# --- Step 4: Update layout to include both layers ---
fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='equirectangular'
    ),
    title_x=0.5
)

fig.show()


/opt/anaconda3/envs/dataviz5200/lib/python3.13/site-packages/_plotly_utils/basevalidators.py:2596: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [31]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# --- Step 1: Prepare your data ---
# Assuming df is already loaded and contains your dataset

# Create a datetime column for easier handling
df['date'] = pd.to_datetime(df[['year', 'month', 'day']])

# --- Step 2: Create the base choropleth map ---
# Fix: Use 'total_affected' instead of 'total.killed' to match your dataframe columns
fig = px.choropleth(
    df_grouped,
    locations='ISO',
    locationmode='ISO-3',
    color='total.affected',  # Changed from 'total.killed' to match your dataframe
    hover_name='country',
    animation_frame='year',
    color_continuous_scale='Reds',
    title='Total Humanitarian Workers Affected by Country (1997–2025)'
)

# --- Step 3: Get the top 20 attacks by total affected ---
# Fix: Use 'total_affected' instead of 'total.affected' to match your dataframe columns
top20 = df.sort_values(by='total.affected', ascending=False).head(20).copy()

# --- Step 4: Normalize marker sizes based on total_affected ---
max_affected = top20['total.affected'].max()
min_affected = top20['total.affected'].min()
size_range = (10, 40)  # min and max marker sizes

top20['marker_size'] = size_range[0] + (
    (top20['total.affected'] - min_affected) / 
    (max_affected - min_affected) * 
    (size_range[1] - size_range[0])
)

# --- Step 5: Process year frames with cumulative attacks ---
years = sorted(df['year'].unique())
frames = []

for year in years:
    # Create frame data for the choropleth
    frame_data = df_grouped[df_grouped['year'] == year]
    
    # Create a choropleth frame for this year
    choropleth = go.Choropleth(
        locations=frame_data['ISO'],
        z=frame_data['total.affected'],  # Changed to match your column name
        locationmode='ISO-3',
        colorscale='Reds',
        showscale=True
    )
    
    # Get attacks that happened up to this year
    year_attacks = top20[top20['year'] <= year].copy()
    
    # For each attack, determine its color based on how old it is
    color_data = []
    for _, attack in year_attacks.iterrows():
        years_ago = year - attack['year']
        
        # Attacks that just happened this year are bright red
        if years_ago == 0:
            color = 'rgba(255, 0, 0, 0.9)'  # Bright red
        else:
            # Older attacks fade from red to black over time
            fade_factor = min(1.0, years_ago / 5)  # Full fade to black over 5 years
            r = int(255 * (1 - fade_factor))  # Red component decreases with age
            opacity = 0.9 - (0.5 * fade_factor)  # Opacity slightly decreases with age
            color = f'rgba({r}, 0, 0, {opacity})'
        
        color_data.append(color)
    
    year_attacks['color'] = color_data
    
    # Create scatter trace for the attacks
    scatter = go.Scattergeo(
        lon=year_attacks['longitude'],
        lat=year_attacks['latitude'],
        mode='markers',
        marker=dict(
            size=year_attacks['marker_size'],
            color=year_attacks['color'],
            line=dict(width=1, color='rgba(255, 255, 255, 0.5)')
        ),
        text=[
            f"Date: {row['month']}/{row['day']}/{row['year']}<br>"
            f"Country: {row['country']}<br>"
            f"Location: {row['city'] if pd.notna(row['city']) else 'Unknown'}<br>"
            f"Total Affected: {row['total.affected']}<br>"
            f"Killed: {row['total.killed']}<br>"
            f"Wounded: {row['total.wounded']}<br>"
            f"Kidnapped: {row['total.kidnapped']}<br>"
            f"Years Ago: {year - row['year']}"
            for _, row in year_attacks.iterrows()
        ],
        hoverinfo='text',
        name=f'Attacks through {year}'
    )
    
    # Create frame with both the choropleth and scatter plot
    frame = go.Frame(
        data=[choropleth, scatter],
        name=str(year)
    )
    frames.append(frame)

# --- Step 6: Update the initial figure with the first frame data ---
# Add the scatter trace for the first year
initial_year = min(years)
initial_attacks = top20[top20['year'] <= initial_year].copy()

if not initial_attacks.empty:
    # All initial attacks are shown in bright red
    fig.add_trace(go.Scattergeo(
        lon=initial_attacks['longitude'],
        lat=initial_attacks['latitude'],
        mode='markers',
        marker=dict(
            size=initial_attacks['marker_size'],
            color='rgba(255, 0, 0, 0.9)',  # Bright red
            line=dict(width=1, color='rgba(255, 255, 255, 0.5)')
        ),
        text=[
            f"Date: {row['month']}/{row['day']}/{row['year']}<br>"
            f"Country: {row['country']}<br>"
            f"Location: {row['city'] if pd.notna(row['city']) else 'Unknown'}<br>"
            f"Total Affected: {row['total_affected']}<br>"
            f"Killed: {row['total.killed']}<br>"
            f"Wounded: {row['total.wounded']}<br>"
            f"Kidnapped: {row['total.kidnapped']}"
            for _, row in initial_attacks.iterrows()
        ],
        hoverinfo='text',
        name=f'Attacks through {initial_year}'
    ))

# --- Step 7: Add animation controls ---
fig.frames = frames

# Update layout with improved animation controls
fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='equirectangular'
    ),
    title_x=0.5,
    updatemenus=[{
        'type': 'buttons',
        'showactive': False,
        'buttons': [{
            'label': 'Play',
            'method': 'animate',
            'args': [None, {
                'frame': {'duration': 800, 'redraw': True},
                'fromcurrent': True,
                'transition': {'duration': 300, 'easing': 'quadratic-in-out'}
            }]
        }, {
            'label': 'Pause',
            'method': 'animate',
            'args': [[None], {
                'frame': {'duration': 0, 'redraw': False},
                'mode': 'immediate',
                'transition': {'duration': 0}
            }]
        }]
    }],
    # Add a slider to select the year
    sliders=[{
        'active': 0,
        'steps': [
            {
                'method': 'animate',
                'label': str(year),
                'args': [[str(year)], {
                    'frame': {'duration': 300, 'redraw': True},
                    'mode': 'immediate',
                    'transition': {'duration': 300}
                }]
            } 
            for year in years
        ],
        'transition': {'duration': 300},
        'x': 0.1,
        'len': 0.9
    }]
)

# Add an annotation to explain the visualization
fig.add_annotation(
    x=0.01,
    y=0.01,
    xref="paper",
    yref="paper",
    text="Circle size represents total affected people.<br>Newer attacks are bright red, older attacks fade to black.",
    showarrow=False,
    font=dict(size=10),
    align="left",
    bgcolor="rgba(255, 255, 255, 0.7)",
    bordercolor="black",
    borderwidth=1,
    borderpad=4
)

# Show the figure
fig.show()

/opt/anaconda3/envs/dataviz5200/lib/python3.13/site-packages/_plotly_utils/basevalidators.py:2596: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [4]:
df['Date'] = pd.to_datetime(df[['year', 'month']].assign(Day=1))

import pandas as pd
import altair as alt


def create_viz2():
    # Load your data
    df = pd.read_csv('../data/security_incidents.csv')
    df = df.clean_names()
    df.columns = df.columns.str.replace('_', '.', regex=False)
    
    # Create full date column from Year and Month
    df['Date'] = pd.to_datetime(df[['year', 'month']].assign(Day=1))

    # Filter out invalid rows
    df = df.dropna(subset=['Date'])

    # Define selectable y-axis variable
    y_dropdown = alt.binding_select(options=['total.affected', 'total.killed', 'total.wounded', 'total.kidnapped'],
                                    name='Y-Axis:')
    y_selection = alt.selection_point(fields=['yvar'], bind=y_dropdown, value='total.affected')

    # Melt your dataframe into long format for flexible plotting
    df_long = df.melt(id_vars=['Date', 'country'], value_vars=['total.affected', 'total.killed', 'total.wounded', 'total.kidnapped'], 
                      var_name='yvar', value_name='yval')

    base = alt.Chart(df_long).transform_filter(
        y_selection
    ).mark_circle(opacity=0.6).encode(
        x=alt.X('Date:T', title="Date"),
        y=alt.Y('yval:Q', title=''),
        color=alt.Color('country:N', legend=None),
        tooltip=['country', 'Date', 'yval', 'yvar']
    ).add_params(
        y_selection
    ).properties(
        width=800,
        height=400,
        title="Interactive Timeline of Humanitarian Impacts"
    ).interactive()

    return base

import altair as alt
alt.data_transformers.disable_max_rows()

chart = create_viz2()
chart


/var/folders/vs/3j3cdp2s1nn0lr6x50_9ycj00000gn/T/ipykernel_97487/1742252758.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df[['year', 'month']].assign(Day=1))
/var/folders/vs/3j3cdp2s1nn0lr6x50_9ycj00000gn/T/ipykernel_97487/1742252758.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df[['year', 'month']].assign(Day=1))


alt.Chart(...)